In [1]:
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import pairwise_distances

warnings.filterwarnings('ignore')

# Toggle to include or exclude already purchased products in recommendations
include_purchased = True

In [2]:
# Load dataset
df = pd.read_csv("stockist_data.csv")

# Define product columns (binary purchase indicators)
product_cols = [
    "AIS(Air Insulated Switchgear)", "RMU(Ring Main Unit)", "PSS(Compact Sub-Stations)",
    "VCU(Vacuum Contactor Units)", "E-House", "VCB(Vacuum Circuit Breaker)",
    "ACB(Air Circuit Breaker)", "MCCB(Moduled Case Circuit Breaker)",
    "SDF(Switch Disconnectors)", "BBT(Busbar Trunking)", "Modular Switches"
]


In [3]:
# Split data into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Save train and test sets
train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)

print("Train and test data saved successfully.")


Train and test data saved successfully.


In [4]:
# Convert training data to binary (bool)
df_products_train = train_df[product_cols].astype(bool)

# Compute Jaccard similarity between product columns
df_products_np = df_products_train.values
jaccard_product_sim = 1 - pairwise_distances(df_products_np.T, metric="jaccard")

# Create a similarity DataFrame
product_similarity_df = pd.DataFrame(jaccard_product_sim, index=product_cols, columns=product_cols)

# Preview
print(product_similarity_df.head())


                               AIS(Air Insulated Switchgear)  \
AIS(Air Insulated Switchgear)                       1.000000   
RMU(Ring Main Unit)                                 0.350498   
PSS(Compact Sub-Stations)                           0.351396   
VCU(Vacuum Contactor Units)                         0.333861   
E-House                                             0.384615   

                               RMU(Ring Main Unit)  PSS(Compact Sub-Stations)  \
AIS(Air Insulated Switchgear)             0.350498                   0.351396   
RMU(Ring Main Unit)                       1.000000                   0.348993   
PSS(Compact Sub-Stations)                 0.348993                   1.000000   
VCU(Vacuum Contactor Units)               0.320513                   0.349515   
E-House                                   0.346090                   0.383446   

                               VCU(Vacuum Contactor Units)   E-House  \
AIS(Air Insulated Switchgear)                     0.3338

In [5]:
def get_top3_products(product_name):
    """Returns top 3 most similar products for a given product using Jaccard similarity."""
    if product_name not in product_similarity_df.index:
        return ["Product not found"]
    return list(product_similarity_df[product_name].sort_values(ascending=False)[1:4].index)


In [6]:
# Binary matrix to help check purchased products per partner
item_matrix = test_df.set_index("Partner_id")[product_cols].astype(bool)


In [7]:
recommendations = []
similarity_scores = []

for index, row in test_df.iterrows():
    partner_id = row["Partner_id"]
    purchased_products = [product for product in product_cols if row[product] == 1]

    if not purchased_products:
        recommendations.append([])
        similarity_scores.append([])
        continue

    recommended_products = set()
    product_scores = []

    for product in purchased_products:
        top_products = get_top3_products(product)
        recommended_products.update(top_products)
        scores = product_similarity_df.loc[product, top_products].values
        product_scores.extend(scores)

    recommended_products = list(recommended_products)

    if not include_purchased:
        already_purchased = user_product_matrix.loc[partner_id]
        recommended_products = [prod for prod in recommended_products if not already_purchased[prod]]

    final_recommendations = recommended_products[:3]
    final_scores = product_scores[:3]

    recommendations.append(final_recommendations)
    similarity_scores.append(final_scores)

    if index < 5:
        print(f"\nRecommendations for Partner {partner_id}:")
        print(f"Purchased Products: {purchased_products}")
        print(f"Recommended Products: {final_recommendations}")
        print(f"Similarity Scores: {final_scores}")



In [8]:
recommendations = []
similarity_scores = []

for index, row in test_df.iterrows():
    partner_id = row["Partner_id"]
    purchased_products = [product for product in product_cols if row[product] == 1]

    if not purchased_products:
        recommendations.append([])
        similarity_scores.append([])
        continue

    recommended_products = set()
    product_scores = []

    for product in purchased_products:
        top_products = get_top3_products(product)
        recommended_products.update(top_products)
        scores = product_similarity_df.loc[product, top_products].values
        product_scores.extend(scores)

    recommended_products = list(recommended_products)

    if not include_purchased:
        already_purchased = user_product_matrix.loc[partner_id]
        recommended_products = [prod for prod in recommended_products if not already_purchased[prod]]

    final_recommendations = recommended_products[:3]
    final_scores = product_scores[:3]

    recommendations.append(final_recommendations)
    similarity_scores.append(final_scores)

    if index < 5:
        print(f"\nRecommendations for Partner {partner_id}:")
        print(f"Purchased Products: {purchased_products}")
        print(f"Recommended Products: {final_recommendations}")
        print(f"Similarity Scores: {final_scores}")


In [9]:
# Add recommendations to test set
test_df["Recommended_Products"] = recommendations
test_df["Similarity_Scores"] = similarity_scores

# Select relevant columns for final output
recommended_df = test_df[["Partner_id", "Recommended_Products", "Similarity_Scores"]]

# Display and save
print("\nFinal Partner Product Recommendations:")
print(recommended_df.head())

output_file = "Partner_Product_Recommendations.csv"
recommended_df.to_csv(output_file, index=False)

print(f"\n Product recommendations saved to {output_file} (Include Purchased: {include_purchased})")



Final Partner Product Recommendations:
    Partner_id                               Recommended_Products  \
521     522_20  [SDF(Switch Disconnectors), VCB(Vacuum Circuit...   
737      738_5  [SDF(Switch Disconnectors), VCB(Vacuum Circuit...   
740      741_1  [SDF(Switch Disconnectors), AIS(Air Insulated ...   
660     661_86  [SDF(Switch Disconnectors), AIS(Air Insulated ...   
411     412_58  [SDF(Switch Disconnectors), AIS(Air Insulated ...   

                                     Similarity_Scores  
521  [0.3846153846153846, 0.36935483870967745, 0.36...  
737  [0.38344594594594594, 0.36142625607779577, 0.3...  
740  [0.3733333333333333, 0.3573825503355704, 0.350...  
660  [0.3733333333333333, 0.3573825503355704, 0.350...  
411  [0.3846153846153846, 0.36935483870967745, 0.36...  

 Product recommendations saved to Partner_Product_Recommendations.csv (Include Purchased: True)


Evaluation Code

In [10]:
#Import Required Libraries
import pandas as pd

In [11]:
# Load test data with one-hot encoded product columns
test_df = pd.read_csv("test_data.csv")

In [12]:
# Load recommendation output (Top-N recommendations per partner)
recommendations_df = pd.read_csv("Partner_Product_Recommendations.csv")

In [13]:
# Fix column naming inconsistency if needed
recommendations_df = recommendations_df.rename(columns={"Partner_ID": "Partner_id"})

In [14]:
#Identify Producy column
meta_cols = [
    'Partner_id', 'Stockist_Type', 'Scheme_Type', 'Sales_Value_Last_Period',
    'Sales_Quantity_Last_Period', 'MRP', 'Growth_Percentage', 'Discount_Applied',
    'Bulk_Purchase_Tendency', 'New_Stockist', 'Feedback_Score'
]
product_cols = [col for col in test_df.columns if col not in meta_cols]


In [15]:
#Convert Product Columns to Purchased List
# ------------------------------
test_df[product_cols] = test_df[product_cols].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

# Purchased product list per partner
test_df['Purchased_Products'] = test_df[product_cols].apply(
    lambda row: [prod for prod, val in zip(product_cols, row) if val == 1], axis=1
)

# Remove non-buyers
test_df['has_purchase'] = test_df['Purchased_Products'].apply(lambda x: len(x) > 0)


In [16]:
# Ensure list parsing from string if needed
recommendations_df["Recommended_Products"] = recommendations_df["Recommended_Products"].apply(eval)

# Merge with test dataset
df_all = pd.merge(
    test_df[['Partner_id', 'Purchased_Products', 'has_purchase']],
    recommendations_df[['Partner_id', 'Recommended_Products']],
    on='Partner_id',
    how='left'
)

In [17]:
# Fill any missing recommendations with empty list
df_all['Recommended_Products'] = df_all['Recommended_Products'].apply(
    lambda x: x if isinstance(x, list) else []
)

In [18]:
results = []

for k in [1, 2, 3]:
    precision_list = []
    recall_list = []

    for _, row in df_all.iterrows():
        actual_set = set(row["Purchased_Products"])
        recommended_k = row["Recommended_Products"][:k]

        if not actual_set:
            continue  # skip if no purchases

        tp = len(set(recommended_k) & actual_set)
        precision = tp / k
        recall = tp / len(actual_set)

        precision_list.append(precision)
        recall_list.append(recall)

    avg_precision = round(sum(precision_list) / len(precision_list), 4) if precision_list else 0
    avg_recall = round(sum(recall_list) / len(recall_list), 4) if recall_list else 0
    f1_k = round(2 * avg_precision * avg_recall / (avg_precision + avg_recall), 4) if (avg_precision + avg_recall) else 0

    results.append({        "Top-K": k,
        "Avg Precision": avg_precision,
        "Avg Recall": avg_recall,
        "Avg F1 Score": f1_k
    })


In [19]:
#Display Results

print("===== Top-K Recommendation Evaluation (Corrected) =====")
for r in results:
    print(f"\nTop-{r['Top-K']}")
    print(f"  Avg Precision : {r['Avg Precision']}")
    print(f"  Avg Recall    : {r['Avg Recall']}")
    print(f"  Avg F1 Score  : {r['Avg F1 Score']}")

===== Top-K Recommendation Evaluation (Corrected) =====

Top-1
  Avg Precision : 0.435
  Avg Recall    : 0.0561
  Avg F1 Score  : 0.0994

Top-2
  Avg Precision : 0.45
  Avg Recall    : 0.1177
  Avg F1 Score  : 0.1866

Top-3
  Avg Precision : 0.4117
  Avg Recall    : 0.1604
  Avg F1 Score  : 0.2309
